# dreal examples

In [1]:
import dreal

In [60]:
# Check defining the variables.
dims = 2
x = [dreal.Variable(f"x[{i}]") for i in range(dims)]

In [61]:
# Check defining the constraints.
constraint_box = dreal.And(x[0] >= 0, x[0] <= 1, x[1] >= 0, x[1] <= 1)
constraint_box

<Formula "((x[0] >= 0) and (x[1] >= 0) and (x[0] <= 1) and (x[1] <= 1))">

In [63]:
# Check defining the constraints with different syntax.
c = dreal.Variable("x")
d = dreal.Variable("y")

constraints = dreal.And(c >= 0, c <= 1, d >= 0, d <= 1)
constraints

<Formula "((x >= 0) and (y >= 0) and (x <= 1) and (y <= 1))">

In [65]:
# Check defining the formula.
formulus1 = dreal.And(x[0] + x[1] >= 1, x[0] - x[1] ** 2 <= 0.3)
print(formulus1)

(((x[0] + x[1]) >= 1) and ((x[0] - pow(x[1], 2)) <= 0.29999999999999999))


In [64]:
# Check defining the formula with eval and expression
expression = "x[0] + x[1] >= 1"
formulus2 = dreal.And(eval(expression), x[0] >= 0, x[0] <= 1, x[1] >= 0, x[1] <= 1)
print(formulus2)

((x[0] >= 0) and (x[1] >= 0) and ((x[0] + x[1]) >= 1) and (x[0] <= 1) and (x[1] <= 1))


In [68]:
# Check assiging a box
box = dreal.Box(x)
box[x[0]] = dreal.Interval(0, 1)
box[x[1]] = dreal.Interval(0, 1)

In [70]:
# Check config
config = dreal.Config()
config.precision = 0.001

In [71]:
result = dreal.CheckSatisfiability(formulus2, config)
print(result)

x[0] : [0.49951171875, 0.5]
x[1] : [0.5, 0.50048828125]


# Interpret function symbolic expression to dreal

In [15]:
import re, copy
from src.test_functions import *

from dreal import sin, cos, exp, log, sqrt, Variable, And, CheckSatisfiability

In [2]:
dims = 2
fn = Ackley(dims)
var_exp, fn_exp = fn.expression()

In [4]:
x = [Variable(f"x[{i}]") for i in range(dims)]

In [38]:
expression = f"{fn_exp} < 1.0"

# Clean up the expression to match dReal syntax
expression = expression.replace(" ", "")
expression = expression.replace("^", "**")
# expression = expression.replace("exp", "dreal.exp")
# expression = expression.replace("sin", "dreal.sin")
# expression = expression.replace("cos", "dreal.cos")
# expression = expression.replace("sqrt", "dreal.sqrt")

to_remove_preceding_sign = True
while to_remove_preceding_sign:
    expression_init = copy.deepcopy(expression)

    expression = re.sub(r"\(\s*\+", "(", expression)
    # expression = re.sub(r"\(\s*\-", "(", expression)

    to_remove_preceding_sign = expression_init != expression

print(expression)

(-(20)*exp(-(0.2)*sqrt(((x[0]**2)+(x[1]**2))/2)))+(-exp((cos((6.283185307179586)*x[0])+cos((6.283185307179586)*x[1]))/2))+((20)+exp(1.0))<1.0


In [39]:
constraint_expression = And(eval(expression))
print(constraint_expression)

((20 + exp(1) - 20 * exp((-0.20000000000000001 * sqrt(((pow(x[0], 2) + pow(x[1], 2)) / 2)))) - exp(((cos((6.2831853071795862 * x[0])) + cos((6.2831853071795862 * x[1]))) / 2))) < 1)


In [40]:
intervals = fn.get_default_domain()

interval_symbol = []
for i in range(dims):
    lb, ub = intervals[i]
    interval_symbol.append(f"x[{i}] >= {lb}")
    interval_symbol.append(f"x[{i}] <= {ub}")

constraint_box = eval(interval_symbol[0])
for i in range(1, len(interval_symbol)):
    constraint_box = And(constraint_box, eval(interval_symbol[i]))

print(constraint_box)

((x[0] >= -32.768000000000001) and (x[1] >= -32.768000000000001) and (x[0] <= 32.768000000000001) and (x[1] <= 32.768000000000001))


In [41]:
constraint_all = And(constraint_box, constraint_expression)
print(constraint_all)

((x[0] >= -32.768000000000001) and (x[1] >= -32.768000000000001) and ((20 + exp(1) - 20 * exp((-0.20000000000000001 * sqrt(((pow(x[0], 2) + pow(x[1], 2)) / 2)))) - exp(((cos((6.2831853071795862 * x[0])) + cos((6.2831853071795862 * x[1]))) / 2))) < 1) and (x[0] <= 32.768000000000001) and (x[1] <= 32.768000000000001))


In [42]:
precision = 0.001
result = CheckSatisfiability(constraint_all, precision)
print(result)

x[0] : [-0.12362229309316736, -0.12262229309316736]
x[1] : [0.069443648113657025, 0.070443648113657026]


In [ ]:
# class dRealExpression:
#     def __init__(self) -> None:
#         self.variables = []
#         self.constraints = []

#     def add_variable(self, name: str):
#         return

#     def intervals_to_constraints(self, intervals: list):
#         """
#         Convert a list of intervals to dReal constraints.

#         Args:
#             intervals (list): A list of intervals [[lb, ub], [lb, ub], ...]
#         Returns:
#             constraints (dreal.Formula): A dReal formula.
#         """

#         assert isinstance(
#             intervals, list
#         ), "Input intervals must be a list [(lb, ub), (lb, ub), ...]"

#         x = self.variables

#         interval_symbol = []
#         for i in range(dims):
#             lb, ub = intervals[i]
#             interval_symbol.append(f"x[{i}] >= {lb}")
#             interval_symbol.append(f"x[{i}] <= {ub}")

#         constraints = eval(interval_symbol[0])
#         for i in range(1, len(interval_symbol)):
#             constraints = dreal.And(constraints, eval(interval_symbol[i]))

#         return constraints